# Lecture 10.2: Iterations and Vectorization
In this lecture we'll learn about:
- [Iteration](#Iteration): `for`/`while` loops.
- [Functional programming](#Functional-programming) (FP): functions that operate on other functions.



In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.3     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



## Iteration
Iteration means, roughly, "running the same piece of code repeatedly". There are many ways to perform iteration in R. The one you have probably heard of is the *for loop*:
```{r}
for (<index> in <vector>) {
    [do something for each value of <index>]
}
```

Note that the loop and the iterator are evaluated within the global environment.



#### while

A while statement can be useful when you aren’t sure how many iterations are needed. 

Here is an example that takes a random walk and terminates if the value is more than 10 units from 0.

#### key words
The following key words are useful within loops:

- break - break out of the currently excuting loop
- next - move to the next iteration immediately, without executing the rest of this iteration (continue in other languages such as C++)

Here is an example using next:



## Conditionals

In programming, we often need to execute a piece of code only if some condition is true. Here are some of the R tools for doing this.



#### if statements

The workhorse for conditional execution in `R` is the if statement.
In the syntax below, note the spacing around the condition enclosed in the parentheses.

Use an else to control the flow without separately checking the condition’s negation:



As you can see above, there is also an ifelse() function that can be useful.

For more complex cases, you may want to check multiple conditions:




### Class Exercise

Write a function to convert a 24-hour time, given as an integer, to a 12-hour time followed by am or pm.  That is, if you input 21, your funciton should output "9pm". 



#### switch

Use a switch when you have multiple discrete options.

Here is an example:

A `switch` can also be used with a numeric expression



### Class Exercises:

1. Read the R code below and determine the value of twos and threes at the end.



2. Read the R code below and determine the value of x at the end.



## Vectorization

Vectorization is a programming concept in which loops are encapsulated within function calls in order to improve the readability of program code.  In high-level interpreted languages such as R, this also is a means of improving efficiency by making use of loops implemented in lower level source code (e.g. C). While loops (`for`, `while`, `apply()`, etc.) have their place in well-written R code,  it is often the case that inefficient or bloated R code can be improved through vectorization.

Vectorization is a programming technique used to avoid explicit loops in order to improve the performance and readability of code.

In its simplest form, vectorization means using built in vectorized functions. That is, rather than looping over a vector to compute its sum, we instead use the vectorized function `sum()`.

The vectorized code is not only easier to type and read but is also faster.



The reason vectorization is faster has to do with the fact that R is an interpreted language and not a compiled one. The difference in time seen here is primarily due to two factors:

R needs to repeatedly interpret what your code means for each iteration of the loop;

each iteration of the for loop in f0 requires indexing into x using the subset function `.[`.

The vectorized `sum()` function also loops over and repeatedly indexes x but it does so in the compiled language C and has been optimized to take advantage of the fact that the elements of a vector are contiguous in memory.

Writing efficient R code requires an understanding of these factors and how vectorization helps to overcome them.